# Mini-Project in NLP

In [2]:
import sys
import os
import zipfile
import re

import pandas as pd
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from IPython.display import display, HTML
from io import StringIO
import seaborn
import matplotlib as plt
from pprint import pprint
import mapply
import multiprocessing
from tqdm import tqdm

from lyricsgenius import Genius
from credentials import CLIENT_ACCESS_TOKEN
from bs4 import BeautifulSoup
import requests

c:\Users\SB\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data collection and cleaning

In [3]:
# tqdm.pandas()
ProgressBar().register() # progress of download process when using dask-dataframe's "apply"

In [4]:
# loading billboard top 100 years 1958-2021
# original dataset from: https://www.kaggle.com/datasets/dhruvildave/billboard-the-hot-100-songs/data

data_directory = "data"
csv_file_name = "charts.csv.zip"
lyrics_csv_file_name = f"lyrics_charts.csv.xz"

csv_file_path = os.path.join(data_directory, csv_file_name)
lyrics_csv_file_path = os.path.join(data_directory, lyrics_csv_file_name)

GENIUS_API_GATEWAY = Genius(
    access_token=CLIENT_ACCESS_TOKEN,
    
    verbose=False,
    timeout=200,
    retries=20,
    sleep_time=3,
    
    skip_non_songs=True,
    remove_section_headers=True,
)

In [5]:
# if not os.path.isfile(lyrics_csv_file_path) and not os.path.isfile(csv_file_path):
#     print("extracted CSV not found")
    
#     with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#         zip_ref.extractall(data_directory)

In [6]:
def covert_date_column(dataframe):
    print(f'{dataframe["date"].dtype = }')

    dataframe["date"] = dataframe["date"].apply(pd.to_datetime, format="%Y-%m-%d")
    print(f'{dataframe["date"].dtype = }')

In [7]:
# if not os.path.isfile(lyrics_csv_file_path):
charts = pd.read_csv(
    csv_file_path, 
    compression="zip", 
    # converters={"date": lambda x: pd.to_datetime(x, format="%Y-%m-%d")}
)

covert_date_column(charts)

dataframe["date"].dtype = dtype('O')
dataframe["date"].dtype = dtype('<M8[ns]')


In [8]:
if not os.path.isfile(lyrics_csv_file_path):
    charts.shape

In [9]:
def print_dataframe(dataframe):
    display(HTML(
        dataframe
        .to_html())
    )

In [10]:
print_dataframe(charts.head(3))
print_dataframe(charts.tail(3))

,date,rank,song,artist,last-week,peak-rank,weeks-on-board
0,2021-11-06,1,Easy On Me,Adele,1.0,1,3
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14


,date,rank,song,artist,last-week,peak-rank,weeks-on-board
330084,1958-08-04,98,Little Serenade,The Ames Brothers,NaN,98,1
330085,1958-08-04,99,I'll Get By (As Long As I Have You),Billy Williams,NaN,99,1
330086,1958-08-04,100,Judy,Frankie Vaughan,NaN,100,1


In [11]:
relevant_columns = ["date", "artist", "song",]
charts = charts[relevant_columns]

In [12]:
splitting_pattern = "|".join((
    r"(?i)[\d,]*Embed$",
    r"\{.*?\}",
    r"see .* live",
    r"get tickets as low as \$[\d.]+",
    # r"|\[.*?\]"
))

def sanitize_lyrics(lyrics):
    # try:
    lyrics = ''.join(re.split(splitting_pattern, lyrics))
    # except Exception as exception:
        # print(f"exception when {lyrics = }")
        # raise exception
    
    lyrics = lyrics.split("Lyrics", maxsplit=1)[1]
    # lyrics = ''.join(lyrics.rsplit("You might also like", maxsplit=1))
    lyrics = ''.join(lyrics.split("You might also like"))
    
    lyrics = re.sub(r"(\n\s*){2,}\n", r"\n\n", lyrics)
    lyrics = lyrics.strip()
    
    return lyrics
    
    
def download_lyrics(artist, title):
    song = GENIUS_API_GATEWAY.search_song(
        artist=artist,
        title=title,
        get_full_info=False,
    )
    return song.lyrics if song is not None else None


def lyrics_from_row(row):
    lyrics = download_lyrics(
        artist=row["artist"], 
        title=row["song"]
    )
    
    return sanitize_lyrics(lyrics) if lyrics is not None else None

In [13]:
def get_n_lines(num_lines, text):
    line_separator = "\n"
    
    return line_separator.join(
        text.split(line_separator)
        [ : num_lines]
    )

In [14]:
# example of downloading lyrics for a single song
if not os.path.isfile(lyrics_csv_file_path):

    # example_song = charts.loc[:, relevant_columns].iloc[-1: , :]
    example_song = charts.loc[ : , relevant_columns].iloc[1:2 , :]
    print_dataframe(example_song)

    raw_lyrics = download_lyrics(example_song.iloc[0].artist, example_song.iloc[0].song)
    num_lines = 10
    print("="*30)
    print("Raw lyrics:")
    print("-"*30)
    print(get_n_lines(num_lines, raw_lyrics))
    print("...")
    print()

    print("="*30)
    print("Sanitized lyrics:")
    print("-"*30)
    sanitized_lyrics = sanitize_lyrics(raw_lyrics)
    print(get_n_lines(num_lines, sanitized_lyrics))
    print("...")

In [15]:
# example of downloading songs for each row in the dataset's dataframe
# TODO isolate a dataframe of wanted songs, download all needed lyrics to dataframe, and save for future use

# if not os.path.isfile(lyrics_csv_file_path):
#     example_songs_copy = charts.loc[:, relevant_columns].iloc[0:3 , :].copy()

#     example_songs_copy["lyrics"] = example_songs_copy.apply(lyrics_from_row, axis=1)

#     example_songs_copy.to_csv(lyrics_csv_file_path, index=False, compression="xz")
# else:
#     example_songs_copy = pd.read_csv(lyrics_csv_file_path, compression="xz")
#     covert_date_column(example_songs_copy)
    
# print_dataframe(example_songs_copy)
# print(example_songs_copy["lyrics"].iloc[1])

In [16]:
# scraping top 100 lists prior to 1958

wikipedia_url = "https://en.wikipedia.org"
wikipedia_billboard_years_url = f"{wikipedia_url}/wiki/Template:Hot_100_year-end_charts"

wikipedia_billboard_years_html = requests.get(wikipedia_billboard_years_url)
billboard_years_html_soup = BeautifulSoup(wikipedia_billboard_years_html.content, 'html.parser')
year_link_tags = billboard_years_html_soup.select("tbody td a")

links_by_year = {int(tag.text): tag.attrs["href"] for tag in year_link_tags}


def full_wikipedia_link(short_wikipedia_link):
    return f'{wikipedia_url}{short_wikipedia_link}'


artist_replacement_pattern = "".join((
    "(?i)(", # case-insensitive
    "|".join((
        " with ",
        " and ",
        "[&,]",
        " x ",
        " ft\.",
        " ft ",
        " feat ",
        " featuring",
        " Co-Starring",
        " Starring",
        " Introducing",
        " Presents",
    )),
    ").*$", # up to the end
))

song_replacement_pattern = "".join((
    "(?i)(", # case-insensitive
    "|".join((
        "\(from .*\)",
        "\(\".*\"\)",
    )),
    ").*$", # up to the end
))


def sanitize_artist_song_columns(table:pd.DataFrame):
    table["artist"] = table["artist"] \
        .str.replace(artist_replacement_pattern, "", regex=True) \
        .str.strip("\" ")
        
    table["song"] = table["song"] \
        .str.replace(song_replacement_pattern, "", regex=True) \
        .str.strip("\" ")
    # table.reset_index(drop=True, inplace=True)
    

def scrap_year(year:int):
    wikipedia_list_url = full_wikipedia_link(links_by_year[year])
    # print(f"following {wikipedia_list_url = }")
    wikipedia_response = requests.get(wikipedia_list_url)
    wikipedia_list_html_soup = BeautifulSoup(wikipedia_response.content, 'html.parser')

    chart_html = wikipedia_list_html_soup.select("table.wikitable.sortable")[0]
    # chart = pd.read_html(StringIO(chart_html.prettify()))[0]
    chart = pd.read_html(StringIO(str(chart_html)))[0]
    chart["year"] = year
    
    # chart.rename(columns={"No.": "rank", "No. (Rank)": "rank"}, inplace=True)
    chart.drop(columns=["No.", "No. (Rank)"], errors="ignore", inplace=True)
    chart.rename(columns={"Title": "song", "Artist(s)": "artist"}, inplace=True)
    
    sanitize_artist_song_columns(chart)
    
    # print_dataframe(chart[chart.index.duplicated()])
    
    # chart.reindex(range(len(chart)))
    
    # assert not chart.index.has_duplicates
    
    return chart


# year = 1950
# print_dataframe(scrap_year(year))

In [17]:
# sanitizing wikipedia-extracted data

# temp = scrap_year(1950)
# print_dataframe(temp)
# sanitize_wikipedia_table(temp)
# print_dataframe(temp)

In [18]:
# %timeit temp["Artist(s)"].apply(lambda artist: re.split("with|&|,", artist)[0]),
# %timeit temp["Artist(s)"].str.extract("^(.*?)(?= with|[&,]|$)"), 
# %timeit temp["Artist(s)"].str.replace("(with |[&,]).*$", "", regex=True),
# %timeit temp["Artist(s)"].str.replace("(with.*$|[&,].*$)", "", regex=True),
# %timeit temp["Artist(s)"].str.replace("(with.*$|&.*$|,.*$)", "", regex=True),
# %timeit temp["Artist(s)"].str.replace("(with.*|&.*|,.*)", "", regex=True),

# print_dataframe(pd.concat([
#             temp["Artist(s)"].apply(lambda artist: re.split("with|&|,", artist)[0]),
#             temp["Artist(s)"].str.extract("^(.*?)(?= with|[&,]|$)"), 
#             temp["Artist(s)"].str.replace("(with |[&,]).*$", "", regex=True),
#             temp["Artist(s)"].str.replace("(with.*$|[&,].*$)", "", regex=True),
#             temp["Artist(s)"].str.replace("(with.*$|&.*$|,.*$)", "", regex=True),
#             temp["Artist(s)"].str.replace("(with.*|&.*|,.*)", "", regex=True),
#         ],
#         axis=1
#     ))

In [19]:
# sanitize_lyrics(download_lyrics(title="The Cry of the Wild Goose", artist="Frankie Laine with Carl T. Fischer"))
# print(sanitize_lyrics(download_lyrics(title=" I Can Dream, Can't I", artist="Andrews Sisters")))

In [20]:
# getting only year-ends' top 100 from downloaded dataset

charts_indexed_by_date = charts.set_index("date")
dates_series = charts["date"]

year_ends = dates_series.groupby(charts_indexed_by_date.index.year).first()
year_ends = year_ends[year_ends.index <= 2020]

# print(charts_index_by_date.index)
print(year_ends)

year_end_charts = charts_indexed_by_date.loc[year_ends]
year_end_charts.index = year_end_charts.index.year
# year_end_charts.index.names = ["year"]
year_end_charts.reset_index(inplace=True)
year_end_charts.rename(columns={"date": "year"}, inplace=True)
year_end_charts

date
1958   1958-12-29
1959   1959-12-28
1960   1960-12-26
1961   1961-12-25
1962   1962-12-29
          ...    
2016   2016-12-31
2017   2017-12-30
2018   2018-12-29
2019   2019-12-28
2020   2020-12-26
Name: date, Length: 63, dtype: datetime64[ns]


,year,artist,song
0,1958,The Chipmunks With David Seville,The Chipmunk Song
1,1958,The Platters,Smoke Gets In Your Eyes
2,1958,The Teddy Bears,"To Know Him, Is To Love Him"
3,1958,Elvis Presley,One Night
4,1958,The Everly Brothers,Problems
...,...,...,...
6294,2020,Niko Moon,Good Time
6295,2020,BRS Kash,Throat Baby (Go Baby)
6296,2020,Lil Baby,Errbody
6297,2020,Carrie Underwood,Favorite Time Of Year


In [21]:
billboard_missing_years = range(min(links_by_year.keys()), year_ends.index.min())
missing_years_dataframes = [scrap_year(year) for year in billboard_missing_years]
missing_years_concatenated = pd.concat(missing_years_dataframes, ignore_index=True)
missing_years_concatenated

,song,artist,year
0,Prisoner of Love,Perry Como,1946
1,To Each His Own,Eddy Howard,1946
2,The Gypsy,The Ink Spots,1946
3,Five Minutes More,Frank Sinatra,1946
4,Rumors Are Flying,Frankie Carle,1946
...,...,...,...
432,Old Cape Cod,Patti Page,1957
433,Mr. Lee,The Bobbettes,1957
434,Blueberry Hill,Fats Domino,1957
435,Whispering Bells,The Del-Vikings,1957


In [22]:
# print(any(x.index.has_duplicates for x in missing_years_dataframes))
# missing_years_concatenated.index.has_duplicates

In [23]:
entirety_of_data = pd.concat([missing_years_concatenated, year_end_charts], ignore_index=True)
entirety_of_data

,song,artist,year
0,Prisoner of Love,Perry Como,1946
1,To Each His Own,Eddy Howard,1946
2,The Gypsy,The Ink Spots,1946
3,Five Minutes More,Frank Sinatra,1946
4,Rumors Are Flying,Frankie Carle,1946
...,...,...,...
6731,Good Time,Niko Moon,2020
6732,Throat Baby (Go Baby),BRS Kash,2020
6733,Errbody,Lil Baby,2020
6734,Favorite Time Of Year,Carrie Underwood,2020


In [24]:
def table_download_lyrics(dataframe):
    dask_dataframe = dd.from_pandas(dataframe, npartitions=len(dataframe))
    dataframe["lyrics"] = dask_dataframe.apply( # parallel execution
        lyrics_from_row, 
        axis=1, 
        meta=(None, str),
    ).compute()

In [25]:
if not os.path.isfile(lyrics_csv_file_path):
    # entirety_of_data["lyrics"] = entirety_of_data.apply(lyrics_from_row, axis=1)
    # entirety_of_data["lyrics"] = entirety_of_data.progress_apply(lyrics_from_row, axis=1)
    
    table_download_lyrics(entirety_of_data)
    entirety_of_data.to_csv(lyrics_csv_file_path, index=False, compression="xz")
else:
    entirety_of_data = pd.read_csv(lyrics_csv_file_path, compression="xz")

print_dataframe(entirety_of_data.head())
print_dataframe(entirety_of_data.tail())

,song,artist,year,lyrics
0,Prisoner of Love,Perry Como,1946,"Alone from night to night you'll find me\nToo weak to break the chains that bind me\nI need no shackles to remind me\nI'm just a prisoner of love!\n\nFor one command I stand and wait now\nFrom one who's master of my fate now\nI can't escape for it's too late now\nI'm just a prisoner of love!\n\nWhat's the good of my caring\nIf someone is sharing those arms with me!\nAlthough she has another\nI can't have another, for I'm not free!\n\nShe's in my dreams awake or sleeping\nUpon my knees to her I'm creeping\nMy very life is in her keeping\nI'm just a prisoner of love\n\nWhat's the good of my caring\nIf someone is sharing those arms with me!\nAlthough she has another\nI can't have another, for I'm not free!\n\nShe's in my dreams awake or sleeping\nUpon my knees to her I'm creeping\nMy very life is in her keeping\nI'm just a prisoner of love"
1,To Each His Own,Eddy Howard,1946,"A rose must remain with the sun and the rain\nOr its lovely promise won't come true\nTo each his own, to each his own\nAnd my own is you\nWhat good is a song if the words just don't belong?\nAnd a dream must be a dream for two\nNo good alone, to each his own\nFor me there's you\n\nIf a flame is to grow there must be a glow\nTo open each door there's a key\nI need you, I know, I can't let you go\nYour touch means too much to me\nTwo lips must insist on two more to be kissed\nOr they'll never know what love can do\nTo each his own, I've found my own\nOne and only you\n\nTwo lips must insist on two more to be kissed\nOr they'll never know what love can do\nTo each his own, I've found my own\nOne and only you"
2,The Gypsy,The Ink Spots,1946,"In a quaint caravan\nThere's a lady they call the Gypsy\nShe can look in the future\nAnd drive away all your fears\nEverything will come right\nIf you only believe the Gypsy\nShe could tell at a glance\nThat my heart was so full of tears\nShe looked at my hand and told me\nMy lover was always true\nAnd yet in my heart I knew, dear\nSomebody else was kissing you\nBut I'll go there again\n'Cause I want to believe the Gypsy\nThat my lover is true\nAnd will come back to me someday\n\nYou see, she looked at my hand and told me that my baby would always be true\nAnd yet, in my heart I knew, dear\nThat somebody else was kissing you\n\nBut I'll go there again\n'Cause I want to believe the Gypsy\nThat my lover is true\nAnd will come back to me someday"
3,Five Minutes More,Frank Sinatra,1946,"Dear, this evening seemed to go so awfully fast\nWe had so much fun and now you're home at last\nI look forward to a kiss or two at the garden gate\nBut she gave me just a peck and insisted it was late\nGive me five minutes more, only five minutes more\nLet me stay, let me stay in your arms\nHere am I, begging for only five minutes more\nOnly five minutes more of your charms\nAll week long I dreamed about our Saturday date\nDon't you know that Sunday morning you can sleep late?\nGive me five minutes more, only five minutes more\nLet me stay, let me stay in your arms\nAll week long I dreamed about our Saturday date\nDon't you know that Sunday morning you can sleep late?\nGive me five minutes more, only five minutes more\nLet me stay, let me stay in your arms\nAh, come on"
4,Rumors Are Flying,Frankie Carle,1946,"Rumors are flying\nThat you've got me sighing\nThat I'm in a crazy kind of a daze\nA lazy sort of a haze\nWhen I go walking\nI hear people talking\nThey say our affair is not just a passing phase\nAnd they whisper 'bout the flowers\nYou keep sending me by the dozen\nAnd they wonder 'bout the hours\nThat you spend with me, it keeps them buzzin'\nRumors are flying\nAnd I'm not denying\nThat people are sure I'm falling in love with you\n'Cause for a change, darling\nAll the rumors are true\n------ guitar solo ------\nAnd they whisper all about the flowers\nThe many little flowers you keep sending me by the dozen\nAnd they wonder all about the hours\nThat you spend with your baby, wha

In [26]:

print(download_lyrics(artist="Kid Cudi", title="Beautiful Trip"))
print("=" * 30)
print(sanitize_lyrics(download_lyrics(title="Beautiful Trip", artist="Kid Cudi")))

47 ContributorsTranslationsEspañolNorsk (bokmål / riksmål)PortuguêsРусскийBeautiful Trip Lyrics
Oh, oh
Oh
Three (Three, two, one)See Kid Cudi LiveGet tickets as low as $47You might also like3Embed
Oh, oh
Oh
Three (Three, two, one)


In [27]:
# print(missing_years_concatenated.index.has_duplicates)
# print(year_end_charts.index.has_duplicates)
# print(entirety_of_data.index.has_duplicates)

In [31]:
# entirety_of_data.loc[entirety_of_data.lyrics.str.contains("Get tickets")]
with_lyrics = entirety_of_data[entirety_of_data.lyrics.notna()]
# with_lyrics[with_lyrics.lyrics.str.contains("Get tickets")].year.hist()
print_dataframe(with_lyrics[with_lyrics.lyrics.str.contains("See .* Live")].head())
print(with_lyrics[with_lyrics.lyrics.str.contains("See .* Live")].iloc[0].lyrics)

,song,artist,year,lyrics
110,Baby Face,Art Mooney,1948,"Ya playin'\nThrowin' that name all up on me\nYa playin'\nKnowin' damn well that you want me\nI'm tired of waitin', conversatin'\nWhy you playin' game, games?\nMakin' love up in the club\nGive a fuck about your name, name\nYa playin'\nThrowin' that name all up on me\nYa playin'\nKnowin' damn well that you want me\nI'm tired of waitin', conversatin'\nWhy you playin' game, games?\nMakin' love up in the club\nGive a fuck about your name, name (Look, ha)\n\nWhat you playin' for?\nI can see it in your eyes, you ain't 'bout that shit\nI'm pelican fly, I left the weather inside\nI got it rainin' in this bitch\nDollar after dollar\nDon't play with a nigga, hop up on this dick\nOne drunk night will change your life\nNow you famous, bitch\nOkay, I'm off that Ace and that Rosé\nJosé, no way\nI'm swervin'\nWith the white CÎROC and that OJ\nMolly? I think I know her face\nI know them titties real but her ass fake\nI keep runnin' with the money at a fast face\nSo baby girl, just meet me at the valet\nTell your girls to pick your car up\nTell her you rollin' with me\nAnd you gon' call that bitch tomorrow\nFirst thing in the mornin', cookin' me breakfast\nYou wearin' my necklace\nSo, girl, why the fuck you out here flexin'?\nFuckin' with these peasants? What?\nSee Chris Brown LiveGet tickets as low as $60\nYa playin'\nThrowin' that name all up on me\nYa playin'\nKnowin' damn well that you want me\nI'm tired of waitin', conversatin'\nWhy you playin' game, games?\nMakin' love up in the club\nGive a fuck about your name, name\nYa playin'\nThrowin' that name all up on me\nYa playin'\nKnowin' damn well that you want me\nI'm tired of waitin', conversatin'\nWhy you playin' game, games?\nMakin' love up in the club\nGive a fuck about your name, name"
410,Diana,Paul Anka,1957,"I'm so young and you're so old\nThis, my darling, I've been told\nI don't care just what they say\n'Cause forever I will pray\nYou and I will be as free\nAs the birds up in the trees\n\nOh, please stay by me, Diana\n\nThrills I get when you hold me close\nOh, my darling, you're the most\nI love you but do you love me?\nOh, Diana, can't you see?\nI love you with all my heart\nAnd I hope we will never part\n\nOh, please stay by me, Diana\n\nOh, my darling, oh, my lover\nTell me that there is no other\nI love you with my heart\nOh-oh, oh-oh, oh don't you know I love you so\nSee Paul Anka LiveGet tickets as low as $42\nOnly you can-a take my heart\nOnly you can-a tear it apart\nWhen you hold me in your loving arms\nI can feel you giving all your charms\nHold me, darling, hold me tight\nSqueeze me baby with all your might\n\nOh, please stay by me, Diana\n\nOh, please, Diana\nOh, please, Diana\nOh, please, Diana"
420,It's Not for Me to Say,Johnny Mathis,1957,"It's not for me to say, you love me\nIt's not for me to say, you'll always care\nOh.. but here for the moment\nI can hold you fast\nAnd press your lips to mine\nAnd dream that love will last\nAs far as I can see, this is heaven\nAnd speaking just for me, It's ours to share\nPerhaps the glow\nOf love will grow\nWith every passing day\nOr we may never meet again\nBut then\nIt's not for me to say\n(Break)\nAnd speaking just for me\nIt's ours to share\nPerhaps the glow\nOf love will grow\nWith every passing day\nOr we may never meet again\nBut then\nIt's not for me to saySee Johnny Mathis LiveGet tickets as low as $47"
425,Chances Are,Johnny Mathis,1957,"Chances are, 'cause I wear a silly grin\nThe moment you come into view\nChances are, you think that I'm in love with you\nJust because my composure sort of slips\nThe moment that your lips meet mine\nChances are, you think my heart's your valentine\n\nIn the magic of moonlight\nWhen I sigh, ""Hold me close, dear""\nChances are, you'll believe the stars\nThat fill the skies are in my eyes\n\nGuess you feel you'll always be\nThe one and only one for me\nAnd if you think you could\nWell, chances are, your chances are awfully good\n\n

Ya playin'
Throwin' that name all up on me
Ya playin'
Knowin' damn well that you want me
I'm tired of waitin', conversatin'
Why you playin' game, games?
Makin' love up in the club
Give a fuck about your name, name
Ya playin'
Throwin' that name all up on me
Ya playin'
Knowin' damn well that you want me
I'm tired of waitin', conversatin'
Why you playin' game, games?
Makin' love up in the club
Give a fuck about your name, name (Look, ha)

What you playin' for?
I can see it in your eyes, you ain't 'bout that shit
I'm pelican fly, I left the weather inside
I got it rainin' in this bitch
Dollar after dollar
Don't play with a nigga, hop up on this dick
One drunk night will change your life
Now you famous, bitch
Okay, I'm off that Ace and that Rosé
José, no way
I'm swervin'
With the white CÎROC and that OJ
Molly? I think I know her face
I know them titties real but her ass fake
I keep runnin' with the money at a fast face
So baby girl, just meet me at the valet
Tell your girls to pick your car

In [32]:
# entirety_of_data2["year"] = entirety_of_data2["year"].apply(pd.to_datetime, format="%Y")
# entirety_of_data2.set_index("year", inplace=True, drop=False)

without_lyrics = entirety_of_data.loc[entirety_of_data.lyrics.isna()]
sanitize_artist_song_columns(without_lyrics)
without_lyrics

,song,artist,year,lyrics
26,Hey! Ba-Ba-Re-Bop,Tex Beneke,1946,NaN
42,Peg o' My Heart,The Harmonicats,1947,NaN
47,Peg o' My Heart,The Three Suns,1947,NaN
55,Temptation (Tim-Tayshun),Red Ingle,1947,NaN
88,Twelfth Street Rag,Pee Wee Hunt,1948,NaN
...,...,...,...,...
6480,Happy Xmas (War Is Over),John,2018,NaN
6492,Close To Me,Ellie Goulding,2018,NaN
6497,Arms Around You,XXXTENTACION,2018,NaN
6585,It's Beginning To Look A Lot Like Christmas,Perry Como,2019,NaN


In [33]:
table_download_lyrics(without_lyrics)

[########################################] | 100% Completed | 202.04 s


In [34]:
without_lyrics

,song,artist,year,lyrics
26,Hey! Ba-Ba-Re-Bop,Tex Beneke,1946,None
42,Peg o' My Heart,The Harmonicats,1947,None
47,Peg o' My Heart,The Three Suns,1947,None
55,Temptation (Tim-Tayshun),Red Ingle,1947,None
88,Twelfth Street Rag,Pee Wee Hunt,1948,None
...,...,...,...,...
6480,Happy Xmas (War Is Over),John,2018,"Happy Christmas, Kyoko\nHappy Christmas, Julia..."
6492,Close To Me,Ellie Goulding,2018,Even though we both know we're liars\nAnd we s...
6497,Arms Around You,XXXTENTACION,2018,"Oh-oh (Ooh, ooh)\n(Lil Pump\nMaluma, baby)\nOh..."
6585,It's Beginning To Look A Lot Like Christmas,Perry Como,2019,It's beginning to look a lot like Christmas\nE...


In [80]:
# try to show that number of songs without found lyrics is relatively small
# maybe find lyrics by hand

print(f'{without_lyrics.query("lyrics.isna()").year.value_counts().sum()}')
print(f'{without_lyrics.query("lyrics.isna()").year.value_counts().idxmax()}')
print(f'{100 * without_lyrics.query("year <= 1960 and lyrics.isna()").year.value_counts(normalize=True)}')
print(f'{without_lyrics.query("year <= 1960 and lyrics.isna()").year.value_counts()}')
without_lyrics.query("year <= 1960 and lyrics.isna()")


134
1960
year
1958    16.326531
1960    16.326531
1953    12.244898
1956    10.204082
1952     8.163265
1959     8.163265
1947     6.122449
1948     6.122449
1949     6.122449
1946     2.040816
1950     2.040816
1951     2.040816
1954     2.040816
1955     2.040816
Name: proportion, dtype: float64
year
1958    8
1960    8
1953    6
1956    5
1952    4
1959    4
1947    3
1948    3
1949    3
1946    1
1950    1
1951    1
1954    1
1955    1
Name: count, dtype: int64


,song,artist,year,lyrics
26,Hey! Ba-Ba-Re-Bop,Tex Beneke,1946,None
42,Peg o' My Heart,The Harmonicats,1947,None
47,Peg o' My Heart,The Three Suns,1947,None
55,Temptation (Tim-Tayshun),Red Ingle,1947,None
88,Twelfth Street Rag,Pee Wee Hunt,1948,None
120,(I'd Like to Get You on a) Slow Boat to China,Kay Kyser,1948,None
123,The Dicky-Bird Song,Freddy Martin,1948,None
151,Whispering Hope,Jo Stafford,1949,None
153,Careless Hands,Sammy Kaye Orchestra,1949,None
155,The Hucklebuck,Tommy Dorsey Orchestra,1949,None


In [79]:
# vietnam war lasted from 1955-11-1 to 1975-4-30

print(f'{len(entirety_of_data.query("lyrics.notna() and year < 1955"))}')
print(f'{len(entirety_of_data.query("lyrics.notna() and 1955 <= year <= 1975"))}')
print(f'{len(entirety_of_data.query("lyrics.notna() and 1975 < year"))}')

283
1805
4399


In [57]:
print(download_lyrics(artist="tex beneke", title="Hey Ba Ba Re Bop"))

37 ContributorsUlysses (Chap. 15 - Circe) LyricsThe Mabbot street entrance of nighttown, before which stretches an uncobbled transiding set with skeleton tracks, red and green will-o'-the-wisps and danger signals. Rows of flimsy houses with gaping doors. Rare lamps with faint rainbow fans. Round Rabaiotti's halted ice gondola stunted men and women squabble. They grab wafers between which are wedged lumps of coal and copper snow. Sucking, they scatter slowly. Children. The swancomb of the gondola, highreared, forges on through the murk, white and blue under a lighthouse. Whistles call and answer.
THE CALLS Wait, my love, and I'll be with you.

THE ANSWERS Round behind the stable.
(A deaf mute idiot with goggle eyes, his shapeless mouth dribbling, jerks past, shaken in Saint Vitus' dance. A chain of children's hands imprisons him.)
THE CHILDREN Kithoguel Salute.
THE IDIOT (Lifts a palsied left arm and gurgles.) Grhahute!
THE CHILDREN Where's the great light?
THE IDIOT (Gobbing.) Ghaghahe

In [ ]:
# we minimized the amount of not found lyrics

not_found_or_instrumental = abc[abc.lyrics.isna()].query("year < 1958")
100 * not_found_or_instrumental.year.value_counts().sum() / entirety_of_data.query("year < 1958").year.value_counts().sum()

## Data processing 

In [63]:
import spacy

In [65]:
NLP_API = spacy.load("en_core_web_sm")

In [67]:
text = entirety_of_data.iloc[-2].lyrics
text

"Yeah\n\nBreak out the tinsel, unpack the lights\nWe're 'bout to bring on the merry and bright\nI hear those jingle bells in the air\nRinging out the sound of joy everywhere\n\n'Tis the season we've been waiting for\n(Waiting, don't keep me waiting)\nCan't you feel it knocking at your door?\n(Knock, knock, it's knocking at your door)\n\nLove pouring out like the snow from the sky\nSnuggled up together sitting by the fire\nOh, it's magical, it's my favorite time\nPretty little wishes tied up in a bow\nAnd every little kiss is like we're underneath the mistletoe\nIt's music to my ears\nChristmas is my favorite time of year\n\nStockings on chimneys, angels on trees\nSugar and cinnamon, it's the little things\nBlankets and movies in black and white\nTogether they make up this colorful life\nSee Carrie Underwood LiveGet tickets as low as $104\n'Tis the season we've been waiting for\n(Waiting, don't keep me waiting)\nOh, can't you feel it knocking at your door?\n(Knock, knock, it's knocking 

In [70]:
doc = NLP_API(text)
doc

Yeah

Break out the tinsel, unpack the lights
We're 'bout to bring on the merry and bright
I hear those jingle bells in the air
Ringing out the sound of joy everywhere

'Tis the season we've been waiting for
(Waiting, don't keep me waiting)
Can't you feel it knocking at your door?
(Knock, knock, it's knocking at your door)

Love pouring out like the snow from the sky
Snuggled up together sitting by the fire
Oh, it's magical, it's my favorite time
Pretty little wishes tied up in a bow
And every little kiss is like we're underneath the mistletoe
It's music to my ears
Christmas is my favorite time of year

Stockings on chimneys, angels on trees
Sugar and cinnamon, it's the little things
Blankets and movies in black and white
Together they make up this colorful life
See Carrie Underwood LiveGet tickets as low as $104
'Tis the season we've been waiting for
(Waiting, don't keep me waiting)
Oh, can't you feel it knocking at your door?
(Knock, knock, it's knocking at your door)

Love pouring o

In [82]:
for token in doc[:5]:
    print(token)

Yeah



Break
out
the
